In [1]:
# from model_functions import *
# from param_functions import *
import numpy as np
import pandas as pd
from IPython import display as ipd
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm
import string
mpl.rcParams['figure.dpi'] = 400

%matplotlib qt


Bad key text.latex.preview in file C:\Users\Adrien\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\Adrien\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\Adrien\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.')
You pr

----------

## Noise Spectrum Plots

In [2]:
### GENERATING DATA

##Constants
h = 6.626e-34
k_B = 1.381e-23
c = 3e8

## Distributions for different noise spectra
NPS = lambda T, f: 1 / ( np.exp( h*f / (k_B*T) ) - 1 )                          # noise photons per second
Planck = lambda T, f: h*f / ( np.exp( h*f / (k_B*T) ) - 1 )                     # noise power
Planck_norm = lambda T, f: ( h*f / (k_B*T) ) / ( np.exp( h*f / (k_B*T) ) - 1 )  # noise power (norm.)    
Planck_3d = lambda T, f: 2*(h*f**3/c**2) / ( np.exp( h*f / (k_B*T) ) - 1 )      # 3d blackbody, in W/(sr m^2 Hz)

## Setting the frequency range, temperatures and setting labels
f = np.logspace(2, 16, 1000)
# T = [300, 35, 2.85, 0.882, 0.082, 0.006]
T = [300, 46, 3.94, 1.227, 0.150, 0.02]
# labels = ['RT', '50K', '4K', 'Still', 'CP', 'MXC']
labels = ['300K', '46K', '3.94K', '1.23K', '150mK', '20mK']

## Calculating the spectra
nps_data = [NPS(t, f) for t in T]               # noise photons per second
data = [Planck(t, f) for t in T]                # noise power spectral density
data_norm = [Planck_norm(t, f) for t in T]      # normalised noise power
data_3d = [Planck_3d(t, f) for t in T]          # 3d black body
data_3d_norm = [d / np.max(d) for d in data_3d] # 3d black body normalised

C:\Users\Adrien\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':
C:\Users\Adrien\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Adrien\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Adrien\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':


In [3]:
### PLOTTING

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(6.5, 8.5))

# setting colours for each distribution
c = ['#FF0000', '#FF6400', '#FFCB37', '#1ADA00', '#00D5E6', '#0042E6']

# plotting noise power noise power 
for i in range(len(T)): 
    axs[0].plot(f, data[i], c=c[i])

# marking the critical point for each temperature (h*omega) / (k_B*T)
axs[0].vlines([k_B*t/h for t in T], [1e-26, 1e-25, 1e-24, 1e-23, 1e-22, 1e-21][::-1], [1e-24, 1e-23, 1e-22, 1e-21, 1e-20, 1e-19][::-1], colors=c, linestyle='dashed', alpha=0.6)

# plotting noise power (norm.)
for i in range(len(T)): 
    axs[1].plot(f, data_norm[i], c=c[i])

axs[1].vlines([k_B*t/h for t in T], 0.4, 0.7, colors=c, linestyle='dashed', alpha=0.6)

# plotting noise photons
for i in range(len(T)): 
    axs[2].plot(f, nps_data[i], c=c[i])

# marking the critical point for each temperature (h*omega) / (k_B*T)
axs[2].vlines([k_B*t/h for t in T], 2, 0.1, colors=c, linestyle='dashed', alpha=0.6)

#setting yaxis scales and limits
axs[0].set_yscale('log')
axs[2].set_yscale('log')
axs[0].set_ylim((1e-33, 1e-18))
axs[0].set_yticks(np.logspace(-33, -18, 6), minor=False)
axs[2].set_ylim((1e-6, 1e9))

# setting x scales and marking qubit transition frequency
for ax in axs:
    ax.set_xscale('log')
    ax.set_xlim((1e3, 1e15))
    ax.set_xticks(np.logspace(3, 15, 13), minor=True)
    # ax.set_xticks([10**i for i in range(3, 15+1, 3)], minor=False)
    # ax.tick_params(axis='x', which='major', width=1)
    ax.grid(True, 'both', 'both')
    rec = mpl.patches.Rectangle((4e9, ax.get_ylim()[0]), 4e9, ax.get_ylim()[1]-ax.get_ylim()[0], facecolor='k', lw=2, alpha=0.2, zorder=-1)
    ax.add_patch(rec)

axs2 = axs[0].twinx()
axs2.set_yscale('log')
axs2.set_ylim(np.array(axs[0].get_ylim())*4/50)
axs2.set_yticks(np.array(axs[0].get_yticks()[1::])*4/50/8)
axs2.set_ylabel('Noise Current Spectral Density,\n' + r'$I^2(\omega)$, $(\mathrm{A}^2/\mathrm{Hz})$', rotation=90)

axs[-1].set_xlabel(r'Frequency, $\omega/2\pi$ (Hz)')

axs[0].set_ylabel('Noise Power Spectral Density,\n' + r'$N(\omega)$, (W/Hz)')
axs[1].set_ylabel('Noise Power Spectral Density,\n' + r'$N(\omega)/N_\mathrm{max}$')
axs[2].set_ylabel('Noise Photons, ' + r'$\bar{n}(\omega)$' + '\n' + r'($\mathrm{s}^{-1}\mathrm{Hz}^{-1}$)')

# axs[0].set_xticklabels

# plt.yscale('linear')
# plt.ylim((0, 1e9))
# # plt.yticks(np.logspace(-6, 9, 6))

# plt.yticks(np.logspace(-6, 9, 6))

# plt.title('Bose-Einstein Distribution')
# plt.ylabel(r'Noise Photons per second ($s^{-1}$)')
# plt.xlabel('Frequency (Hz)')
# plt.vlines(5.75e9, 0,1, 'grey', alpha=0.45, lw=9)

# adding the lettering
for n, ax in enumerate(axs.flatten('F')):
    ax.text(-0.05, 1.1, string.ascii_lowercase[n]+')', 
    transform=ax.transAxes, 
    size=12, weight='bold')
axs[1].legend(labels)
plt.tight_layout()

# plt.savefig('21.10.28/Noise Spectra.png', dpi=400)

plt.show()

## Noise Saturation

In [5]:
h = 6.626e-34
k_B = 1.381e-23
c = 3e8

n_dist = lambda T, f: 1 / ( np.exp( h*f / (k_B*T) ) - 1 )                          # noise photons per second
I_dist = lambda T: 4*k_B*T/50                  # noise power

T = [46, 3.94, 1.227, 0.150, 0.02]
# labels = ['RT', '50K', '4K', 'Still', 'CP', 'MXC']
labels = ['46K', '3.94K', '1.23K', '150mK', '20mK']

A = np.linspace(0, 60, 200)
n = lambda a, t: n_dist(300, 6e9)*10**(-a/10) + (1-10**(-a/10))*n_dist(t, 6e9)
I = lambda a, t: I_dist(300)*10**(-a/10) + (1-10**(-a/10))*I_dist(t)

n_data = [[n(a, t) for a in A] for t in T]
I_data = [[I(a, t) for a in A] for t in T]

n_sat = [n_dist(t, 6e9) for t in T[:-1]]
I_sat = [I_dist(t) for t in T]

In [7]:
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(6, 6))

c = ['#FF0000', '#FF6400', '#FFCB37', '#1ADA00', '#00D5E6', '#0042E6']

for i in range(len(n_data)-1, -1, -1):
    axs[0].plot(A, n_data[i], c=c[i+1])

# axs[0].hlines(n_sat, 0, 60, linestyle='dashed', linewidth=0.75)

for i in range(len(I_data)-1, -1, -1):
    axs[1].plot(A, I_data[i], c=c[i+1])

# axs[1].hlines(I_sat, 0, 60, linestyle='dashed', linewidth=0.75)

for ax in axs:
    ax.set_yscale('log')
    locmin = mpl.ticker.LogLocator(base=10.0,subs=[0.1*(i+1) for i in range(9)],numticks=12)
    ax.yaxis.set_minor_locator(locmin)
    ax.grid(True, 'major', 'both')
    axs2 = ax.twinx()
    axs2.set_yscale('log')
    axs2.set_ylim(ax.get_ylim())

axs[0].set_ylabel('Noise Photons, ' + r'$\bar{n}$'+ ' ' + r'($\mathrm{s}^{-1}\mathrm{Hz}^{-1}$)')
axs[1].set_ylabel('Noise Current, ' + r'$I^2$' + r'($\mathrm{A}^{2}/\mathrm{Hz}$)')
axs[-1].set_xlabel('Attenuation (dB)')
axs[0].legend(fridge_ours['labels'])

for n, ax in enumerate(axs.flatten('F')):
    ax.text(-0.05, 1.05, string.ascii_lowercase[n]+')', 
    transform=ax.transAxes, 
    size=12, weight='bold')

plt.tight_layout()

# plt.savefig('21.10.28/NoiseSaturation.png', dpi=400)

plt.show()

In [13]:
T = [300, 46, 3.94, 1.227, 0.150, 0.02]
# labels = ['RT', '50K', '4K', 'Still', 'CP', 'MXC']
labels = ['300K', '46K', '3.94K', '1.23K', '150mK', '20mK']

A = np.linspace(0, 40, 200)
n = lambda a, t1, t2: n_dist(t1, 6e9)*10**(-a/10) + (1-10**(-a/10))*n_dist(t2, 6e9)
I = lambda a, t1, t2: I_dist(t1)*10**(-a/10) + (1-10**(-a/10))*I_dist(t2)

n_data2 = [[n(a, T[i-1], T[i]) for a in A] for i in range(1, len(T))]
I_data2 = [[I(a, T[i-1], T[i]) for a in A] for i in range(1, len(T))]

In [14]:
fig, axs = plt.subplots(2, 2, sharex=True, figsize=(6.5, 8))

c = ['#FF0000', '#FF6400', '#FFCB37', '#1ADA00', '#00D5E6', '#0042E6']

for i in range(len(n_data)-1, -1, -1):
    axs[0, 0].plot(A, n_data[i], c=c[i+1])

for i in range(len(I_data)-1, -1, -1):
    axs[1, 0].plot(A, I_data[i], c=c[i+1])

for i in range(len(n_data2)-1, -1, -1):
    axs[0, 1].plot(A, n_data2[i], c=c[i+1])

for i in range(len(I_data2)-1, -1, -1):
    axs[1, 1].plot(A, I_data2[i], c=c[i+1])

for ax in axs.flatten():
    ax.set_yscale('log')

# axs[0, 1].set_xlabel()

plt.show()

## Noise through sequence of attenuators

In [5]:
## GENERATING DATA

## Constants
h = 6.626e-34
k_B = 1.381e-23
c = 3e8

# Distributions
n_dist = lambda T, f: 1 / ( np.exp( h*f / (k_B*T) ) - 1 )       # noise photons distribution
P_dist = lambda T, f: h*f / ( np.exp( h*f / (k_B*T) ) - 1 )     # noise power distribution
I_dist = lambda T: 4*k_B*T/50                                   # low frequency current distribution

# Setting temperatures
T = [46, 3.94, 1.227, 0.150, 0.02]
# labels = ['RT', '50K', '4K', 'Still', 'CP', 'MXC']
labels = ['46K', '3.94K', '1.23K', '150mK', '20mK']

# Setting frequency range AND atttenuation distribution
f = np.logspace(3, 15, 1000)
A = [20, 20, 20, 20, 20]
# n = lambda a, t: n_dist(300, 6e9)*10**(-a/10) + (1-10**(-a/10))*n_dist(t, 6e9)
# P = lambda a, t: P_dist(300, 6e9)*10**(-a/10) + (1-10**(-a/10))*n_dist(t, 6e9)
# I = lambda a, t: I_dist(300)*10**(-a/10) + (1-10**(-a/10))*I_dist(t)

# Calcaulting the noise power generated by each stage and the outgoing noise on each stage
P_stage = [P_dist(t, f) for t in T]
P_out = [P_dist(300, f)]
for i in range(len(T)):
    # formula to determine outgoing noise power based on impinging noise and noise generated by stage
    P_out.append(P_out[i]*10**(-A[i]/10)+(1-10**(-A[i]/10))*P_stage[i])

# Calculating the noise photons generated by each stage and the outgoing noise on each stage
n_stage = [n_dist(t, f) for t in T]
n_out = [n_dist(300, f)]
for i in range(5):
    # formula to determine outgoing noise power based on impinging noise and noise generated by stage
    n_out.append(
        n_out[i]*10**(-A[i]/10) + (1-10**(-A[i]/10))*n_stage[i]
        )

# n_data = [[n(a, t) for a in A] for t in T]
# I_data = [[I(a, t) for a in A] for t in T]

# n_sat = [n_dist(t, 6e9) for t in T[:-1]]
# I_sat = [I_dist(t) for t in T]

C:\Users\Adrien\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Adrien\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


In [6]:
# PLOTTING INCOMING, OUTGOING NOISE AND NOISE GENERATED BY EACH STAGE, FOR EACH STAGE
fig, axs = plt.subplots(6, 2, sharex=True, figsize=(10, 9))

c = ['#FF0000', '#FF6400', '#FFCB37', '#1ADA00', '#00D5E6', '#0042E6']

rec1 = mpl.patches.Rectangle((4e9, 1e-4), 4e9, 1e4-1e-4, facecolor='k', lw=2, alpha=0.2, zorder=-1)
rec2 = mpl.patches.Rectangle((4e9, 1e-27), 4e9, 1e-19-1e-27, facecolor='k', lw=2, alpha=0.2, zorder=-1)

ylim1 = (1e-8, 1e2)
ylim2 = (1e-32, 1e-19)

yticks1 = [1e-6, 1e-3, 1e0]
yticks2 = [1e-30, 1e-27, 1e-24, 1e-21]

# INCOMING, OUTGOING, AND STAGE NOISE POWER
for i in range(5):
    axs[i, 0].plot(f, n_out[i])     #incoming noise power
    axs[i, 0].plot(f, n_stage[i])   #stage noise power
    axs[i, 0].plot(f, n_out[i+1])   #outgoing noise power

    #formatting
    axs[i, 0].set_xscale('log')
    axs[i, 0].set_yscale('log')
    axs[i, 0].set_xlim((1e3, 1e15))
    axs[i, 0].set_ylim(ylim1)
    rec1 = mpl.patches.Rectangle((4e9, ylim1[0]), 4e9, ylim1[1]-ylim1[0], facecolor='k', lw=2, alpha=0.2, zorder=-1)
    axs[i, 0].add_patch(rec1)
    axs[i, 0].set_yticks(yticks1)
    axs[i, 0].set_xticks(np.logspace(3, 15, 13), minor=True)
    axs[i, 0].set_xticks([10**i for i in range(3, 15+1, 3)], minor=False)
    # ax.set_xticklabels([], minor=True)
    axs[i, 0].grid(True, 'major', 'both')

axs[2, 0].set_ylabel('Noise Photons,\n' + r'$\bar{n}(\omega)$ ($\mathrm{s}^{-1}\mathrm{Hz}^{-1}$)')

# INCOMING, OUTGOING, AND STAGE NOISE POWER
for i in range(5):
    axs[i, 1].plot(f, P_out[i])     #incoming noise power
    axs[i, 1].plot(f, P_stage[i])   #stage noise power
    axs[i, 1].plot(f, P_out[i+1])   #outgoing noise power

    #formatting
    axs[i, 1].set_xscale('log')
    axs[i, 1].set_yscale('log')
    axs[i, 1].set_xlim((1e3, 1e15))
    axs[i, 1].set_ylim(ylim2)
    rec2 = mpl.patches.Rectangle((4e9, ylim2[0]), 4e9, ylim2[1]-ylim2[0], facecolor='k', lw=2, alpha=0.2, zorder=-1)
    axs[i, 1].add_patch(rec2)
    axs[i, 1].set_yticks(yticks2)
    axs[i, 1].set_xticks(np.logspace(3, 15, 13), minor=True)
    axs[i, 1].set_xticks([10**i for i in range(3, 15+1, 3)], minor=False)
    # ax.set_xticklabels([], minor=True)
    axs[i, 1].grid(True, 'major', 'both')
    axs2 = axs[i, 1].twinx()
    axs2.set_yscale('log')
    axs2.set_ylim(np.array(ylim2)*4/50)
    axs2.set_yticks(np.array(yticks2+[1e-18])*4/50/8)
    if i == 2: axs2.set_ylabel('Noise Current\nSpectral Density,\n' + r'$I^2(\omega)$, $(\mathrm{A}^2/\mathrm{Hz})$')
axs[2, 1].set_ylabel('Noise Power\nSpectral Density,\n' + r'$N(\omega)$, (W/Hz)')

#formatting
for ax in axs[5, :].flatten():
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim((1e3, 1e15))
    ax.set_xticks(np.logspace(3, 15, 13), minor=True)
    ax.set_xticks([10**i for i in range(3, 15+1, 3)], minor=False)
    ax.set_xticklabels([], minor=True)
    ax.grid(True, 'major', 'both')
    ax.set_xlabel(r'Frequency, $\omega/2\pi$ (GHz)')

axs[5, 0].plot(f, n_out[0], c=c[0])
for i in range(5): axs[5, 0].plot(f, n_stage[i], c=c[i+1])
axs[5, 0].plot(f, n_out[-1], c='k', linewidth=2)

axs[5, 1].plot(f, P_out[0], c=c[0])
for i in range(5): axs[5, 1].plot(f, P_stage[i], c=c[i+1])
axs[5, 1].plot(f, P_out[-1], c='k', linewidth=2)
    
axs[5, 0].set_ylim(ylim1)
axs[5, 1].set_ylim(ylim2)
axs[5, 0].set_yticks(yticks1)
axs[5, 1].set_yticks(yticks2)

rec1 = mpl.patches.Rectangle((4e9, ylim1[0]), 4e9, ylim1[1]-ylim1[0], facecolor='k', lw=2, alpha=0.2, zorder=-1)
rec2 = mpl.patches.Rectangle((4e9, ylim2[0]), 4e9, ylim2[1]-ylim2[0], facecolor='k', lw=2, alpha=0.2, zorder=-1)
axs[5, 0].add_patch(rec1)
axs[5, 1].add_patch(rec2)
axs2 = axs[5, 1].twinx()
axs2.set_yscale('log')
axs2.set_ylim(np.array(ylim2)*4/50)
axs2.set_yticks(np.array(yticks2+[1e-18])*4/50/8)

axs[2, 0].legend(['Incident', 'From Stage', 'Outgoing'], fontsize=8)
# axs[2, 1].legend(['Incident', 'From Stage', 'Outgoing'])
axs[5, 0].legend(['RT']+labels+['At Device'], fontsize=8, loc=(-0.5, -0.01))
# axs[]

#lettering
for n, ax in enumerate(axs.flatten('F')):
    ax.text(-0.1, 0.95, string.ascii_lowercase[n]+')', 
    transform=ax.transAxes, 
    size=12, weight='bold')

plt.tight_layout()

# plt.savefig('21.10.28/NoiseDownFridge.png', dpi=400)

plt.show()

## Attenuator Dissipation

In [10]:
A = [20, 0, 20, 0, 20]
P = [1e-3*10**(0)]
P_diss = []

for i in range(5):
    P_diss.append((1-10**(-A[i]/10)) * P[i])
    P.append(10**(-A[i]/10) * P[i])

In [36]:
fig, axs = plt.subplots(1, 1, figsize=(4, 6))

axs.barh([-i*3 for i in range(5)], P[1:], height=1, color='b')
axs.barh([-i*3 for i in range(5)], P_diss, height=1, left=P[1:], color='r')
axs.set_xscale('log')
axs.set_xlim(1e-10, 1e-2)
axs.set_xticks([10**i for i in range(-10, -1)], minor=True)
axs.set_xticks([10**i for i in range(-9, -1, 2)], minor=False)
axs.set_xticklabels([], minor=True)

axs2 = axs.twiny()
axs2.set_xscale('log')
axs2.set_xlim(np.array(axs.get_xlim())*4/50)
axs2.set_xticks([10**i*4/50/8 for i in range(-9, -1)], minor=True)
axs2.set_xticks(np.array(axs.get_xticks())*4/50/8, minor=False)
axs2.set_xticklabels([], minor=True)

axs2.set_xlabel(r'Current Squared, $I^2$ $(\mathrm{A}^2)$')

axs.set_yticks([-i*3 for i in range(5)])
axs.set_yticklabels(fridge_ours['labels'])
axs.grid(True, 'major', 'x', alpha=0.4)
axs.set_xlabel(r'Power, $P$ (W)')
axs.legend(['Transmitted', 'Dissipated'])

plt.tight_layout(True)

plt.savefig('21.10.28/AttenuatorDissipation.png', dpi=400)

plt.show()

In [13]:
P, P_diss

([0.001, 1e-05, 1e-05, 1.0000000000000001e-07, 1.0000000000000001e-07, 1e-09],
 [0.00099, 0.0, 9.9e-06, 0.0, 9.900000000000001e-08])

## Cable Dissipation - Drive

In [2]:
coax_co_params['219-SS-SS']['att_4'](6)

6.380000000000001

In [16]:
att = np.array([0, 20, 0, 20, 20])
c_att = coax_co_params['219-SS-SS']['att_4'](6) * np.array(fridge_ours['lengths'])
total_att = np.sum(att + c_att)

p_in = 1e-3*10**(-5/10)

p = [p_in]
for i in range(0, 4):
    p.append(p[i]*10**(-(att+c_att)[i]/10))

a_ac = np.array(p) * (1 - 10**(-c_att/10))
# a_ac = pd.Series(a_ac, index=fridge_ours['labels'])
a_att = np.array(p) * (1 - 10**(-att/10))
a_up = a_ac[1:].tolist() + [0]
a_down = a_ac
a_split = (a_up+a_down)/2
a_inner = []
acc = 0
for i in range(5):
    if att[i] > 0:
        a_inner.append(a_ac[i])
        acc = 0
    else:
        a_inner.append(0)
        acc += a_ac[i]


In [25]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))

axs.plot([i for i in range(5)], a_att+np.array(a_up))
axs.plot([i for i in range(5)], a_att+np.array(a_down))
axs.plot([i for i in range(5)], a_att+np.array(a_split))
# axs.plot([i for i in range(5)], a_att+np.array(a_inner))
axs.set_yscale('log')
axs.set_xticks([i for i in range(5)])
axs.set_xticklabels(fridge_ours['labels'])
axs.set_xlabel('Stage')
axs.set_ylabel('Power Dissipated, $P$ (W)')
axs.set_title('Power Dissipated by Drive Lines')
axs.grid(True, 'major', 'y')

axs.legend(['Upper Stage', 'Lower Stage', 'Split'])

plt.tight_layout(True)

plt.savefig('21.10.28/DriveCableHeat.png', dpi=400)

plt.show()

## Passive Load From three sections

In [3]:
therm_scheme_drive = [[False, True, False, True, True], #inner
                  [False, True, False, True, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_flux = [[False, True, False, False, True], #inner
                  [False, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_output = [[True, True, False, False, True], #inner
                  [True, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

In [5]:
c_drive = '219-SS-SS'

d = coax_co_params[c_drive]['d']
A =  cs_areas['219']

p_drive = passive_load(stage_labels=fridge_ours['labels'], 
        diameters=coax_co_params[c_drive]['d'],
        lengths=fridge_ours['lengths'],
        therm_cond=[coax_co_params[c_drive]['tc'], lambda t: 0, coax_co_params[c_drive]['tc']],
        therm_scheme=therm_scheme_drive,
        stage_temps=fridge_ours['temps'])

NameError: name 'therm_scheme_drive' is not defined

In [2]:
c = '219-SS-SS'

A = cs_areas['219']
L = fridge_ours['lengths']
T = [300] + fridge_ours['temps']
s_c = coax_co_params[c]['tc']
s_d = gen_therm_cond(PTFE_fit, 4)

def pazz(a):
    P = []
    acc_d, acc_i = 0, 0
    for i in range(5):
        dat = []
        T_range = np.linspace(T[i+1], T[i], 1000)
        dT = T_range[1]-T_range[0]
        dat.append(A[2]/L[i] * np.sum(s_c(T_range)*dT))
        acc_d += A[1]/L[i] * np.sum(s_d(T_range)*dT) * 10**-2
        acc_i += A[0]/L[i] * np.sum(s_c(T_range)*dT)
        if a[i]:
            dat.append(acc_d)
            dat.append(acc_i)
            acc_d, acc_i = 0, 0
        else:
            dat.append(0)
            dat.append(0)
        P.append(dat)
    return P

A1 = [True, True, True, True, True]
A2 = [True, False, True, False, True]
P1 = np.array(pazz(A1))
P2 = np.array(pazz(A2))

In [6]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(6, 5))

P = [P1, P2]
c = ['r', 'g', 'b']

for i in range(2):
    axs[i].barh([-iii*4-1 for iii in range(5)], np.sum(P[i], axis=1), height=3, alpha=0.5, color='dodgerblue')
    for ii in range(3):
        axs[i].barh([-iii*4-ii for iii in range(5)], P[i][:, ii], height=1, color=c[ii])
    axs[i].set_xscale('log')
    axs[i].set_xlim(10**(-11.5), 10**-1.5)
    axs[i].set_xticks([10**iii for iii in range(-11, -1, 3)], minor=False)
    axs[i].set_xticks([10**iii for iii in range(-11, -1)], minor=True)
    axs[i].set_xticklabels([], minor=True)
    axs[i].set_xlabel(r'Passive Load, $P_\mathrm{passive}$ (W)')
    axs[i].set_yticks([-iii*4-1 for iii in range(5)])
    axs[i].set_yticklabels(fridge_ours['labels'])
    axs[i].grid(True, 'both', 'x', alpha=0.4)

axs[1].legend(['Total', 'Outer Cond.', 'Dielectric', 'Inner Cond.'], fontsize = 9)

for n, ax in enumerate(axs.flatten('F')):
    ax.text(-0.05, 1.02, string.ascii_lowercase[n]+')', 
    transform=ax.transAxes, 
    size=12, weight='bold')

plt.tight_layout(True)

plt.savefig('21.10.28/PassiveLoads.png', dpi=400)

plt.show()

In [22]:
np.sum(P, axis=1)

array([2.05846851e-02, 3.72462636e-04, 3.04818829e-06, 6.44960584e-07,
       6.49402698e-09])

In [18]:
passive_load(stage_labels=fridge_ours['labels'], 
        diameters=coax_co_params[c]['d'],
        lengths=fridge_ours['lengths'],
        therm_cond=[coax_co_params[c]['tc'], lambda t: 0, coax_co_params[c]['tc']],
        therm_scheme=[[True]*5]*3,
        stage_temps=fridge_ours['temps'])


50K      1.994248e-02
4K       3.336059e-04
Still    2.504977e-06
CP       5.293405e-07
MXC      5.330329e-09
Name: Passive Load (W), dtype: float64

## T2 from drive noise

In [2]:
chi = 43e6**2/1e9

h = 6.626e-34
k_B = 1.381e-23
c = 3e8

n_BE = lambda T, f: 1 / ( np.exp( h*f / (k_B*T) ) - 1 )   
T2 = lambda n: 1/(n*(n+1)*(chi)**2)

# T = [300]+fridge_ours['temps']
T_data = np.logspace(np.log10(0.01), np.log10(600), 300)
n_data = [n_BE(t, 6e9) for t in T_data]
T2_data = [T2(n) for n in n_data]

In [3]:
fig, axs = plt.subplots(figsize=(6, 6))
axs.plot(T_data, n_data)
axs.set_xscale('log')
axs.set_yscale('log')

axs2 = axs.twinx()
axs2.plot(T_data, T2_data, c='r')
axs2.set_xscale('log')
axs2.set_yscale('log')

axs.set_xlabel('Temperature')
axs.set_ylabel('Noise Photons')
axs2.set_ylabel('T2')

# axs.set_yticks([1e-1, 1e0, 1e1, 1e2, 1e3])
# axs.set_xlabel('Noise Temperature (K)')
# axs.set_ylabel(r'Dephasing Rate from Current Noise $\Gamma_{\phi}  (s^{-1})$')

# axs2 = axs.twinx()
# axs3 = axs.twiny()

# axs2.set_yscale('log')
# axs2.set_ylim(1 / np.array(axs.get_ylim()))
# axs2.set_yticks([1/i for i in axs.get_yticks()])
# axs2.set_ylabel(r'Maximum $T_2$ (s)')

# axs3.set_xscale('log')
# axs3.set_xlim(S_eff(np.array(axs.get_xlim())))
# # axs3.set_xticks([S_eff(i) for i in axs.get_xticks()])
# axs3.set_xlabel(r'Noise Current, $I^2   (A^2/Hz)$')

# temps = [(7e-3, 20e-3), (70e-3, 150e-3), (3.5, 4), (1.1, 1.227), (45, 50), (290, 300)]
# h = axs.get_ylim()[1] - axs.get_ylim()[0]
# hm = (axs.get_ylim()[1] + axs.get_ylim()[0])/2
# for i in range(6):
#     w = temps[i][1] - temps[i][0]
#     wm = (temps[i][1] + temps[i][0]) / 2
#     b = mpl.patches.Rectangle((wm-w/2, hm-h/2), w, h, facecolor='r', lw=2, alpha=0.5, zorder=-1)
#     axs.add_patch(b)

# axs.annotate('MXC', (0.023, 1e3))
# axs.annotate('CP', (0.153, 0.5e3))
# axs.annotate('Still', (1.25, 0.1e3))
# axs.annotate('4K', (4.2, 0.1e2))
# axs.annotate('50K', (52, 0.3e1))
# axs.annotate('RT', (150, 0.5e0))

# axs.set_title('Noise temperature vs. Qubit Dephasing')
plt.tight_layout(True)

# plt.savefig('21.10.18/Flux Noise Temp.png')

plt.show()

## Energy Distribution of qubit VS LC oscillator

In [26]:
e = [2*(1-np.exp(-0.3*i)) for i in range(1, 6)]
[np.arccos(1-i)for i in e]
e


[0.5183635586365642,
 0.902376727811947,
 1.1868606805188016,
 1.3976115761755956,
 1.5537396797031404]

In [44]:
## qubit
f = lambda x: 1-np.cos(x)
x = np.linspace(-3.5, 3.5, 100)
x1 = np.linspace(-3.75, -3.5, 50)

e = [2*(1-np.exp(-0.3*i)) for i in range(1, 6)]
ex = [np.arccos(1-i)for i in e]

fig, axs = plt.subplots(figsize=(7, 5))
axs.plot(x, f(x), c='b', linewidth=4)
axs.plot(x1, f(x1), linestyle='dashed', c='b', linewidth=2)
axs.plot(-1*x1, f(-1*x1), linestyle='dashed', c='b', linewidth=2)

axs.hlines(e, -1*np.array(ex), ex, linewidth=2)
plt.savefig('21.10.28/Qubitenergies.png', dpi=400)
plt.show()

In [45]:
## LC oscillator
f = lambda x: x**2
x = np.linspace(-1.5, 1.5, 100)
x1 = np.linspace(-1.65, -1.5, 50)

e = [0.4*i for i in range(1, 6)]
ex = [np.sqrt(i) for i in e]

fig, axs = plt.subplots(figsize=(5, 5))
axs.plot(x, f(x), c='b', linewidth=4)
axs.plot(x1, f(x1), linestyle='dashed', c='b', linewidth=2)
axs.plot(-1*x1, f(-1*x1), linestyle='dashed', c='b', linewidth=2)

axs.hlines(e, -1*np.array(ex), ex, linewidth=2)
plt.savefig('21.10.28/LCenergies.png', dpi=400)
plt.show()

## Cable Attenuation Plots

In [2]:
# cablesSS = ['119-SS-SS', '219-SS-SS', '119-AgSS-SS', '219-AgSS-SS']
# cablesCuNi = ['119-CuNi-CuNi', '219-CuNi-CuNi', '119-AgCuNi-CuNi', '219-AgCuNi-CuNi']
# cablesNbTi = ['119-NbTi-NbTi', '219-NbTi-NbTi']

cablesdf = ['219-CuNi-CuNi', '219-SS-SS', '119-CuNi-CuNi', '119-SS-SS']
cablesout = ['219-AgCuNi-CuNi', '219-AgSS-SS', '119-AgCuNi-CuNi', '119-AgSS-SS']
cablessuper = ['119-NbTi-NbTi', '219-NbTi-NbTi'][::-1]

f = [0.5, 1, 5, 10, 19.99]

In [15]:
fig, axs = plt.subplots(2, 3, sharey=True, sharex=True, figsize=(9, 5))

for c in cablesdf:
    # for fi in f:
    #     print(fi)
    #     print(-1*coax_co_params[c]['att_300'](fi))
    d300 = [-1*coax_co_params[c]['att_300'](fi) for fi in f]
    d4 = [-1*coax_co_params[c]['att_4'](fi) for fi in f]
    axs[0, 0].plot(f, d300)
    axs[1, 0].plot(f, d4)

for c in cablesout:
    d300 = [-1*coax_co_params[c]['att_300'](fi) for fi in f]
    d4 = [-1*coax_co_params[c]['att_4'](fi) for fi in f]
    axs[0, 1].plot(f, d300)
    axs[1, 1].plot(f, d4)

for c in cablessuper:
    d300 = [-1*coax_co_params[c]['att_300'](fi) for fi in f]
    d4 = [-1*coax_co_params[c]['att_4'](fi) for fi in f]
    axs[0, 2].plot(f, d300)
    axs[1, 2].plot(f, d4)

for i in range(3):
    axs[1, i].set_xlabel(r'Frequency, $\omega/2\pi$ (GHz)')

# for in range

axs[0, 0].legend(cablesdf, loc=(0.2, 1.05))
axs[0, 1].legend(cablesout, loc=(0.2, 1.05))
axs[0, 2].legend(cablessuper, loc=(0.2, 1.05))
    

axs[0, 0].set_ylabel('Cable Attenuation\nat 300K (dB/m)')
axs[1, 0].set_ylabel('Cable Attenuation\nat 4K (dB/m)')

for n, ax in enumerate(axs.flatten('F')):
    ax.grid(True, 'major', 'both')
    ax.text(-0.05, 1.1, string.ascii_lowercase[n]+')', 
    transform=ax.transAxes, 
    size=12, weight='bold')

plt.tight_layout(True)

plt.savefig('21.11.02/CableAttData.png', dpi=400)

plt.show()

In [40]:
axs

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001543B7AE808>,
      dtype=object)

In [14]:
f

[0.5, 1, 5, 10, 20]

In [13]:
c

'119-SS-SS'

In [10]:
-1*coax_co_params[c]['att_300'](0.5)

-5.3